In [1]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import MidTermFeatures
import os
import numpy as np
from scipy.spatial.distance import euclidean
from scipy.stats import spearmanr

def extract_global_features(filepath):
    [Fs, x] = audioBasicIO.read_audio_file(filepath)
    x = audioBasicIO.stereo_to_mono(x)
    
    # Extraction des attributs à moyen terme
    mid_term_features, short_term_features, mid_feature_names = MidTermFeatures.mid_feature_extraction(x, Fs, 1.0 * Fs, 1.0 * Fs, 0.050 * Fs, 0.025 * Fs)
    
    # Calcul des attributs globaux (moyenne et écart-type des attributs à moyen terme)
    mean_features = np.mean(mid_term_features, axis=1)
    std_features = np.std(mid_term_features, axis=1)
    global_features = np.concatenate((mean_features, std_features))
    
    return global_features, mid_feature_names


C:\Users\Julien\miniconda3\envs\PCD\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:

def process_directory(directory_path):
    audio_files = [f for f in os.listdir(directory_path) if f.endswith('.mp3') or f.endswith('.wav')]
    all_global_features = []
    all_feature_names = None
    
    for audio_file in audio_files:
        filepath = os.path.join(directory_path, audio_file)
        global_features, feature_names = extract_global_features(filepath)
        all_global_features.append(global_features)
        if all_feature_names is None:
            all_feature_names = feature_names
    
    return np.array(all_global_features), all_feature_names, audio_files

In [3]:

def calculate_distances(global_features, selected_features, reference_index=0):
    distances = []
    for i in range(len(global_features)):
        if i != reference_index:
            dist = euclidean(global_features[reference_index][selected_features], global_features[i][selected_features])
            distances.append((i, dist))
    return sorted(distances, key=lambda x: x[1])

def rank_tracks(distances):
    return [x[0] for x in distances]

def evaluate_ranking(sorted_indices, ground_truth):
    rho, _ = spearmanr(sorted_indices, ground_truth)
    return rho

def sequential_forward_selection(global_features, ground_truth, feature_names):
    selected_features = []
    remaining_features = list(range(global_features.shape[1]))
    best_rho = -1

    while remaining_features:
        best_feature = None
        best_new_rho = -1

        for feature in remaining_features:
            current_features = selected_features + [feature]
            distances = calculate_distances(global_features, current_features)
            sorted_indices = rank_tracks(distances)
            rho = evaluate_ranking(sorted_indices, ground_truth)

            if rho > best_new_rho:
                best_new_rho = rho
                best_feature = feature

        if best_new_rho > best_rho:
            best_rho = best_new_rho
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
        else:
            break

    return selected_features, best_rho

In [4]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import MidTermFeatures
import os
import numpy as np
from scipy.spatial.distance import euclidean
from scipy.stats import spearmanr


directory_path = "data"
ground_truth = [1, 2, 3, 4, 5, 6, 7]

global_features, feature_names, audio_files = process_directory(directory_path)

selected_features, best_rho = sequential_forward_selection(global_features, ground_truth, feature_names)

distances = calculate_distances(global_features, selected_features)

sorted_indices = rank_tracks(distances)

print("Attributs sélectionnés:", [feature_names[i] for i in selected_features])
print("Corrélation de Spearman:", best_rho)
print("Classement des morceaux par distance croissante avec le morceau 0:")
for idx in sorted_indices:
    print(f"Morceau {idx} avec distance {dict(distances)[idx]}")
print("Classement de référence:", ground_truth)


Attributs sélectionnés: ['spectral_centroid_mean', 'chroma_2_mean']
Corrélation de Spearman: 1.0
Classement des morceaux par distance croissante avec le morceau 0:
Morceau 1 avec distance 0.0038710584117659192
Morceau 2 avec distance 0.005708265253409976
Morceau 3 avec distance 0.006429584931379759
Morceau 4 avec distance 0.01836278215406612
Morceau 5 avec distance 0.024162628033182185
Morceau 6 avec distance 0.031286143856644064
Morceau 7 avec distance 0.046018519224962855
Classement de référence: [1, 2, 3, 4, 5, 6, 7]
